In [1]:
%%html
<style>.edit_mode div.cell.selected {width: 72rem !important;}</style>
<script>$("div.input:first").hide();</script>

# Advent of Code 2018
http://adventofcode.com/2018

## Day 1
Puzzle input: `input01.txt`
### Part 1

In [1]:
with open('input01.txt') as f:
    freq_change = [int(n) for n in f]
    
print(sum(freq_change))

576


### Part 2

In [2]:
last_freq = 0
freq_history = set()
n = len(freq_change)

i = 0
while last_freq not in freq_history:
    freq_history.add(last_freq)
    last_freq += freq_change[i]    
    i = (i+1) % n
    
print(last_freq)

77674


## Day 2
Puzzle input: `input02.txt`
### Part 1

In [3]:
with open('input02.txt') as f:
    words = [w.strip() for w in f]

In [4]:
from collections import Counter

twos, threes = 0, 0
for w in words:
    c = Counter(w)
    if 2 in c.values(): twos += 1
    if 3 in c.values(): threes += 1
        
print(twos * threes)

5904


### Part 2

In [5]:
from itertools import combinations

def diff_count(w1, w2):
    return sum(a != b for a,b in zip(w1, w2))

def common(w1, w2):
    return ''.join(a for a,b in zip(w1, w2) if a == b)

for w1, w2 in combinations(words, 2):
    if diff_count(w1, w2) == 1: print(common(w1, w2))

jiwamotgsfrudclzbyzkhlrvp


## Day 3
Puzzle input: `input03.txt`
### Part 1

In [6]:
with open('input03.txt') as f:
    claim_raw = [c.strip() for c in f]

In [7]:
from collections import defaultdict
from re import search

claim_regex = r'\#(\d+)\s+\@\s+(\d+),(\d+)\:\s+(\d+)x(\d+)'

claim_id = defaultdict(set)
for c_raw in claim_raw:
    c_id, i, j, m, n = (int(g) for g in search(claim_regex, c_raw).groups())
    for p in range(i, i+m):
        for q in range(j, j+n):
            claim_id[(p,q)].add(c_id)
            
print(sum(len(claims) > 1 for claims in claim_id.values()))

104241


### Part 2

In [8]:
claim_set = set.union(*claim_id.values())

for position, claims in claim_id.items():
    if len(claims) > 1:
        claim_set -= claim_id[position]
                
print(claim_set)

{806}


## Day 4
Puzzle input: `input04.txt`
### Part 1

In [9]:
with open('input04.txt') as f:
    guard_records = [g.strip() for g in f]

In [10]:
from collections import defaultdict
from re import search
from datetime import datetime, timedelta
import pandas as pd

guard_regex1 = '\[.+\] Guard \#(\d+)'
guard_regex2 = '\[(\d+)\-(\d+)\-(\d+)\ (\d+)\:(\d+)\] (.+) .+'

asleep = defaultdict(int)
for gr in sorted(guard_records):
    try:
        grps = search(guard_regex1, gr).groups()
        curr_guard = int(grps[-1])
    except:
        grps = search(guard_regex2, gr).groups()
        yr, mo, da, hr, mi = (int(g) for g in grps[:-1])
        dt2 = datetime(yr, mo, da, hr, mi)
        if 'falls' in grps[-1]: 
            dt1 = dt2
        if 'wakes' in grps[-1]:
            dt = dt1
            while dt < dt2:
                asleep[(curr_guard, dt.minute)] += 1
                dt += timedelta(minutes = 1)
            dt1 = dt2
            
sleep_data = pd.DataFrame([[k[0], k[1], v] for k,v in asleep.items()],
                          columns = ['guard', 'minute', 'asleep'])

strategy1 = sleep_data[
    sleep_data.guard == sleep_data.groupby('guard').sum().asleep.idxmax()
].sort_values('asleep', ascending = False).head(1)

print((strategy1.guard * strategy1.minute).values)

[3212]


### Part 2

In [11]:
strategy2 = sleep_data.groupby(
    ['guard', 'minute'], as_index = False
).sum().sort_values('asleep', ascending = False).head(1)

print((strategy2.guard * strategy2.minute).values)

[4966]
